In [3]:
import torch
from NLPUtils import *
import re

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Entrenamiento:

with open('promptsl40.train','rb') as file:
    train_lines = file.readlines()
    corpus = list(itertools.chain([' '.join(re.findall(r'\w+',line.decode('iso-8859-1'))[1:]) for line in train_lines]))

window_size = 2           # Tamaño de la ventana del contexto.
cutoff_freq = 0           # Palabras con una frecuencia menor o igual a cutoff_freq son excluídas del vocabulario.
batch_size = 512          # Tamaño del batch.

model = 'SkipGram'            # Método de entrenamiento.
embedding_dim = 200       # Dimensión del espacio de los word vectors.
device = 'cuda:1'         # Dispositivo sobre el cual se entrena. 
state_dict = None         # Parámetros pre-entrenados.
paralelize = False        # Flag para decirle al programa que use las 2 gpus

trainer = Word2VecTrainer(corpus, model, cutoff_freq=cutoff_freq,window_size=window_size,batch_size=batch_size)
trainer.InitModel(state_dict=state_dict, device=device, embedding_dim=embedding_dim)

Dispositivo seleccionado: cuda:1


In [29]:
epochs = 150              # Cantidad de epochs
learning_rate = 5e-4      # Tasa de aprendizaje
sample_loss_every = 10    # Calcular la loss cada este número
algorithm = 'Adam'        # Algoritmo de optimización

trainer.Train(algorithm=algorithm, epochs=epochs, sample_loss_every=sample_loss_every, lr=learning_rate)

Starting training...
Optimization method: Adam
Learning Rate: 0.0005
Number of epochs: 150
Running on device (cuda:1)



NotImplementedError: 

In [ ]:
vocab = trainer.dataloader.dataset.vocabulary
forward = lambda x: trainer.model.out(trainer.model.emb(x))
device = trainer.device
log10 = torch.log(torch.tensor(10,dtype=torch.float,device=device))

with open('./lm_files/lm_l40_bigram_train_test', 'rb') as file:
    lines = file.readlines()
    new_lm_file = []
    for line in lines:
        ngram_line = re.search(r'([\-\.\d]+)\t([\w<s></s>]+[ ]?[\w<s></s>]*)([\t]?[\-\.\d]*)\n',line.decode('iso-8859-1'))
        if ngram_line is not None:
            ngram = ngram_line.groups()[1]
            if ' ' in ngram:
                w1, w2 = ngram.split(' ')
                score = forward(torch.tensor(vocab[w1],device=device))
                log_prob = max(((score[vocab[w2]] - torch.logsumexp(score,dim=0))/log10).item(),-99)
                new_line = '{:.6f}\t{}\n'.format(log_prob,ngram)
            else:
                score = forward(torch.tensor(vocab[ngram],device=device))
                log_prob = max(((score[vocab[ngram]] - torch.logsumexp(score,dim=0))/log10).item(),-99)
                new_line = '{:.6f}\t{}\t0\n'.format(log_prob,ngram)
        else:
            new_line = line
        new_lm_file.append(new_line)
    
with open('./lm_files/lm_l40_word_vectors_w2_cbow', 'wb') as file:
    text = ''.join(new_lm_file)
    file.write(text.encode('iso-8859-1'))